Let's code the "occasionally dishonest casino" model.  First, let's define transition probabilities and emission probabilities:

In [ ]:
from __future__ import print_function,division

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)

np.random.seed(0)

# 0-> fair die, 1-> loaded die 
states_possible = [0,1]

# Possible outputs from the die
observations_possible = [0,1,2,3,4,5]

# Transition matrix
A = np.array([[0.99,0.01],[0.03,0.97]])

# Emission probabilities as a function of state
E = np.array([[1./6.]*6,
              [0.1,0.1,0.1,0.1,0.1,0.5]])

Let's simulate 1000 observations given these statistics

In [ ]:
def emit(state):
    roll = np.random.choice(observations_possible,p=E[state])
    return roll

def transition(state):
    return np.random.choice(states_possible,p=A[state])

states = [0]
observations = [emit(states[0])]

m = 1000

for t in range(m-1):
    states.append(transition(states[-1]))
    observations.append(emit(states[-1]))
    
states = np.array(states)
observations = np.array(observations)

plt.plot(states,'r-')
plt.plot(observations,'k.')
plt.show()

Since we know the transition matrix, let's imagine that our initial state probability is given by the stable distribution.

In [ ]:
w,v = np.linalg.eig(A.T)
p_steady = (v[:,0].real)/(v[:,0].real.sum())
print (p_steady)

In [ ]:
def likelihood(x,z):
    return E[z,x]

def prediction(z):
    return np.dot(z,A)

def prior(z):
    return p_steady[z]

Now we can compute our initial state probability using Bayes' rule:

In [ ]:
x_0 = observations[0]

P_0 = likelihood(x_0,states_possible)*prior(states_possible)
P_0/=P_0.sum()
print (P_0)

likelihoods = np.array([likelihood(o,states_possible) for o in observations])

In [ ]:
alpha = np.zeros((m,2))
alpha[0] = P_0
for t in range(1,1000):
    P_previous = alpha[t-1]
    predictions = prediction(P_previous)   
    numerator = likelihoods[t]*predictions
    alpha[t] = numerator/numerator.sum()


In [ ]:
alpha

In [ ]:
plt.plot(states,'r-')
plt.plot(observations,'k.')
plt.plot(alpha[:,1],'g-')
plt.show()

This is pretty good!  If we kept this tally going as we played the game, we could choose to change our bet when the loaded die was in use.  Let's take a closer look:

In [ ]:
plt.plot(states[:300],'r-')
plt.plot(alpha[:300,1],'g-')
plt.show()

Note the lag.  This is because our information flow is one-sided, purely from left to right, however this is unavoidable if we wish to operate in an on-line capacity.  However, let's imagine a different situation in which we were the state gaming board, and we wanted to tell when a dealer was cheating by examining security camera footage.  This is a slightly different problem because now we can condition on future events as well as past ones.  

In [ ]:
T = m-1
beta = np.zeros((m,2))

# Base case: always proportional to one
beta[T] = 1.0
beta[T]/=beta[T].sum()
for t in range(T-1,-1,-1):
    P_next = beta[t+1]
    numerator = np.dot(A,P_next*likelihoods[t+1])
    beta[t] = numerator/numerator.sum()


Now we can multiply the forward probabilities by the reverse probabilities and normalize to get the total probabilities:

In [ ]:
total_probs = alpha*beta
total_probs = total_probs/total_probs.sum(axis=1)[:,np.newaxis]

In [ ]:
#plt.plot(observation_list,'k.')
plt.plot(total_probs[:300,1],'b-',lw=5.0)
plt.plot(alpha[:300,1],'g-',lw=5.0)
plt.plot(states[:300],'r-',lw=5.0)
plt.show()

Now, we turn to the case in which we do not know the model *a priori*, which is to say that we do not know the state transition matrix $A$.  Perhapse we also don't know the emission probabilities.  How do we find these things?  
In the case where we have observations of both the states and the observations, then reconstruction is very easy and proceeds as in the case of the non-hidden Markov model: for the transition matrix, we simply count the transitions from one state to the other, then normalize them afterwards.

In [ ]:
A_approx = np.zeros((2,2))
prior_prob = np.array([np.sum(states==s) for s in states_possible])
prior_prob = prior_prob/float(prior_prob.sum())

for t in range(m-1):
    i = states[t]
    j = states[t+1]
    A_approx[i,j] += 1
    
A_approx = A_approx/A_approx.sum(axis=1)[:,np.newaxis]


In [ ]:
A_approx

Pretty good recovery of the correct matrix.  Now we turn to recovery of the emission probability model.  This is also straightforward: we simply compute the frequency of each data point, given a particular state and place these in a table:
  

In [ ]:
E_approx = np.zeros((2,6))
for z,x in zip(states,observations):
    E_approx[z,x] += 1
    
E_approx = E_approx/E_approx.sum(axis=1)[:,np.newaxis]

In [ ]:
print (E_approx)

This is, of course, very similar to the correct emission probabilities.

With these matrices in hand, it is easy to run the the forward-backward algorithm and infer the state probabilities.

In [ ]:
def likelihood(x,z):
    return E_approx[z,x]

def prediction(z):
    return np.dot(z,A_approx)

def prior(z):
    return prior_prob[z]

x_0 = observations[0]

P_0 = likelihood(x_0,states_possible)*prior(states_possible)
P_0/=P_0.sum()

likelihoods = np.array([likelihood(o,states_possible) for o in observations])

alpha = np.zeros((m,2))
alpha[0] = P_0
for t in range(1,1000):
    P_previous = alpha[t-1]
    predictions = prediction(P_previous)   
    numerator = likelihoods[t]*predictions
    alpha[t] = numerator/numerator.sum()
    
T = m-1
beta = np.zeros((m,2))

# Base case: always proportional to one
beta[T] = 1.0
beta[T]/=beta[T].sum()
for t in range(T-1,-1,-1):
    P_next = beta[t+1]
    numerator = np.dot(A,P_next*likelihoods[t+1])
    beta[t] = numerator/numerator.sum()

total_probs = alpha*beta
total_probs = total_probs/total_probs.sum(axis=1)[:,np.newaxis]

plt.plot(total_probs[:300,1],'b-')
plt.plot(states[:300],'r-')
plt.show()


Unsurprisingly, we do a fairly good job.  Now, however, let's consider the more difficult case in which we only have access to the observations (the dice rolls), but not the states.  Thus we need to learn both the transition matrix and emission probabilities *only* with the rolls.  

This is not dissimilar to a mixture model, with the states as the unobserved latent variables.  As in the case of the mixture model, we utilize the *expectation-maximization* algorithm.  Recall that for EM, we alternate between computing the probability of the states given the current transition matrix and emission probabilities.  We then compute the transition matrix and emission probabilities given the current states.

In [ ]:
x_0 = observations[0]

np.random.seed(42)

# Let's make a very vague guess regarding the transition matrix and emission probs.
#A_0 = np.random.rand(2,2)
#A_0/=A_0.sum(axis=1)[:,np.newaxis]
A_0 = np.array([[0.7,0.3],[0.3,0.7]])

E_0 = np.random.rand(2,6)
E_0/=E_0.sum(axis=1)[:,np.newaxis]
#E_0 = np.array([[1./6.]*6,[1./6.]*6])
A = A_0
E = E_0

ces = []

# Loop over the number of EM iterations:
for f in range(150):
    
    # Compute the prior given the current estimate of A
    w,v = np.linalg.eig(A.T)
    unity_index = np.argmax(w)
    p_steady = (v[:,unity_index].real)/(v[:,unity_index].real.sum())

    # define the likelihood model.
    def likelihood(x,z):
        return E[z,x]

    # define a prediction
    def prediction(z):
        return np.dot(z,A)

    # define the prior
    def prior(z):
        return p_steady[z]

    # Expectation step: Use the forward-backward algorithm to compute the probability distribution of 
    # states, as well as the joint distribution of adjacent states P(z_{t+1},z_t|x_t)

    P_0 = likelihood(x_0,states_possible)*prior(states_possible)
    P_0/=P_0.sum()

    likelihoods = np.array([likelihood(o,states_possible) for o in observations])

    alpha = np.zeros((m,2))
    alpha[0] = P_0
    for t in range(1,1000):
        P_previous = alpha[t-1]
        predictions = np.dot(P_previous,A)  
        numerator = likelihoods[t]*predictions
        alpha[t] = numerator/numerator.sum()
    
    T = m-1
    beta = np.zeros((m,2))

    # Base case: always proportional to one
    beta[T] = 1.0
    beta[T]/=beta[T].sum()
    for t in range(T-1,-1,-1):
        P_next = beta[t+1]
        numerator = np.dot(A,P_next*likelihoods[t+1])
        beta[t] = numerator/numerator.sum()

    gamma = alpha*beta
    gamma = gamma/gamma.sum(axis=1)[:,np.newaxis] 
    
    sigma = np.zeros((m,2,2))
    for t in range(m):
        for i in range(2):
            for j in range(2):
                try:    
                    sigma[t,i,j] = alpha[t,i]*A[i,j]*beta[t+1,j]*likelihoods[t+1,j]
                except IndexError:
                    sigma[t,i,j] = alpha[t,i]*A[i,j]
                 
        sigma[t,:,:] /= sigma[t,:,:].sum()

    # Maximization step: Use the estimate of states to compute the maximum likelihood estimators for the 
    # transition matrix and emission probabilities.
    A = sigma[:-1].sum(axis=0)/gamma[:-1].sum(axis=0)[:,np.newaxis]
    E = np.array([gamma[observations==k].sum(axis=0)/gamma.sum(axis=0) for k in range(6)]).T
    
    cross_entropy = -np.sum(states*np.log(gamma[:,1]) + (1-states)*np.log(1-gamma[:,1]))
    ces.append(cross_entropy)
    #print (A)


In [ ]:
plt.plot(gamma[:,1],'g-')
plt.plot(observations,'k.')
plt.plot(states,'r-')
plt.show()

In [ ]:
plt.plot(ces)
plt.show()